In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd
import time

In [2]:
import re
import requests
from requests_html import HTML
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import DesiredCapabilities
from selenium.common.exceptions import TimeoutException, NoSuchElementException, TimeoutException, ElementClickInterceptedException

In [3]:
def chrome(url):
    # support to get response status and headers
    d = DesiredCapabilities.CHROME
    d['loggingPrefs'] = {'performance': 'ALL'}
    opt = webdriver.ChromeOptions()
#     opt.add_argument("--headless")
#     opt.add_argument("--disable-xss-auditor")
    opt.add_argument("--disable-web-security")
    opt.add_argument("--allow-running-insecure-content")
    opt.add_argument("--no-sandbox")
    opt.add_argument("--disable-setuid-sandbox")
    opt.add_argument("--disable-webgl")
    opt.add_argument("--disable-popup-blocking")
    # prefs = {"profile.managed_default_content_settings.images": 2,
    #          'notifications': 2,
    #          }
    # opt.add_experimental_option("prefs", prefs)
    browser = webdriver.Chrome(options=opt,desired_capabilities=d)
#     browser.add_cookie({"name" : "_session_public", "value" : "EMDkrM2AsLpsnFEu3L9bkz6oS7QUnbVWyXBOW1SpMpozxRwuSTZHwkIxaia0zESLkJUK6BxDmOAv3L7yycPsWi6voEhuZkW7iB7J"})
#     browser.add_cookie({"name" : "city", "value" : "{%22id%22:1%2C%22name%22:%22%D0%90%D0%BB%D0%BC%D0%B0%D1%82%D1%8B%22%2C%22slug%22:%22almaty%22%2C%22is_disabled%22:false%2C%22deposit_rate%22:1%2C%22label%22:%22empty%22%2C%22country_id%22:1%2C%22latitude%22:43.221313%2C%22longitude%22:76.955436}"})
    browser.implicitly_wait(10)
    browser.set_page_load_timeout(20)
    browser.get(url)
    return browser.page_source, browser

In [4]:
# /en/fabric/7137786-genevieve-floral-by-crystal_walen
def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)./?service=(?P<service>\d+)"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['service']

In [6]:
category=[]
list_cat_hrefs=[]
url = 'https://naimi.kz/#/'
req = requests.get(url)
content, browser = chrome(url)
browser.find_element_by_xpath("//button[@class='btn blue mr-3 flex-item']").click()

In [7]:
elements=browser.find_elements_by_class_name("category")
for i in elements:
    elements = i.find_elements_by_tag_name("a")
    for el in elements:
        category.append(el)
        list_cat_hrefs.append(el.get_attribute("href"))

In [8]:
list_cat_hrefs

['https://naimi.kz/almaty/category/domashnij-personal',
 'https://naimi.kz/almaty/category/remontnye-raboty',
 'https://naimi.kz/almaty/category/otdelochnye-raboty',
 'https://naimi.kz/almaty/category/tekhnika-i-oborudovanie',
 'https://naimi.kz/almaty/category/perevozka-i-dostavka',
 'https://naimi.kz/almaty/category/itfreelance',
 'https://naimi.kz/almaty/category/krasotazdorovie',
 'https://naimi.kz/almaty/category/delovieuslugi',
 'https://naimi.kz/almaty/category/repetitor-kurs',
 'https://naimi.kz/almaty/category/volonteri',
 'https://naimi.kz/almaty/category/na-postoyannuy-rabotu']

In [9]:
list_sub_cat_hrefs=[]
for l in list_cat_hrefs:
    req = requests.get(l)
    content, browser = chrome(l)
    cats=browser.find_elements_by_xpath("//a[@class='categories--service blue outline pt-1 pb-1 pl-2 pr-2']")
    for i in cats:
        print(i.get_attribute("href"))
        list_sub_cat_hrefs.append(i.get_attribute("href"))
    
    
    
    
    

https://naimi.kz/almaty/service/uborka
https://naimi.kz/almaty/service/nyani-i-sidelki
https://naimi.kz/almaty/service/domrabotnitsy
https://naimi.kz/almaty/service/sadovniki-i-dvorniki
https://naimi.kz/almaty/service/povar
https://naimi.kz/almaty/service/shvei-i-portnye
https://naimi.kz/almaty/service/uhod-za-zhivotnymi
https://naimi.kz/almaty/service/himchistka
https://naimi.kz/almaty/service/dezinfektsiya
https://naimi.kz/almaty/service/santehniki
https://naimi.kz/almaty/service/elektriki
https://naimi.kz/almaty/service/mebelschiki
https://naimi.kz/almaty/service/dveri-i-zamki
https://naimi.kz/almaty/service/master-na-chas
https://naimi.kz/almaty/service/okna-i-balkony
https://naimi.kz/almaty/service/gazovschik
https://naimi.kz/almaty/service/malyary-i-shtukatury
https://naimi.kz/almaty/service/raznorabochie
https://naimi.kz/almaty/service/raboty-s-plitkoi-i-kafelem
https://naimi.kz/almaty/service/stroitelnye-raboty
https://naimi.kz/almaty/service/remont-pod-klyuch
https://naimi.kz/

WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=86.0.4240.193)


In [10]:
len(list_sub_cat_hrefs)

63

In [ ]:
datas = []
for cat in list_sub_cat_hrefs:
    i=1
    result = None
    text_found=r'Рекомендованные специалисты'
    while text_found is not None:
        try:
            url = '{}?page={}#/'.format(cat,i)
            req = requests.get(url)
            content, browser = chrome(url)
            html_r = HTML(html=content)
            text_found = re.search(r'Рекомендованные специалисты', content)
            if text_found is None:
                break
            list_of_hrefs=[]
            a_elements = []
            content_blocks = browser.find_elements_by_class_name("service-specialists")
            for block in content_blocks:
                elements = block.find_elements_by_tag_name("a")
                for el in elements:
                    a_elements.append(el)
                    list_of_hrefs.append(el.get_attribute("href"))
                    fabric_links = [x for x in list(list_of_hrefs) if x.startswith("https://naimi.kz/specialist/")]
            for path in fabric_links:
                id_, slug_ = extract_id_slug(path)
                print(id_, slug_)
                data = {
                    "id": id_,
                    "slug": slug_,
                    "path": path,
                    "scraped": 0 # True / False -> 1 / 0 
                }
                datas.append(data)
            print('page {}'.format(i))   
            i+=1    
        except:
            break

In [ ]:
datas

In [ ]:
!pipenv install pickle

In [5]:
import pickle

In [ ]:
with open(r'datas.pickle', 'wb') as output:
    pickle.dump(datas, output)

In [6]:
with open(r'datas.pickle', 'rb') as data:
     datas = pickle.load(data)

In [9]:
len(datas[:8000])

8000

In [10]:
len(datas[8000:])

8033

In [ ]:
cookies={1:{"name" : "_session_public", "value" : "PZQNzME5q4YRqIIgzJOEPKvTvlueobznww0DcT9MoagB69SwfIuS1ZX126LurmOnMSK7FhKBGnfK6ykU37MSKoNxMe8EgZyx7En0"},
        2:{"name" : "_session_public", "value" : "PZQNzME5q4YRqIIgzJOEPKvTvlueobznww0DcT9MoagB69SwfIuS1ZX126LurmOnMSK7FhKBGnfK6ykU37MSKoNxMe8EgZyx7En0"},
        3:{"name" : "_session_public", "value" : "PZQNzME5q4YRqIIgzJOEPKvTvlueobznww0DcT9MoagB69SwfIuS1ZX126LurmOnMSK7FhKBGnfK6ykU37MSKoNxMe8EgZyx7En0"}
        }

In [11]:
cookies={1:{"name" : "_session_public", "value" : "5pZTFel7yzs2v45x3aSV5GAXLBIwLKGhAU9DAdJKx5DwBudoiZBTcOxtb59SpTBFYkLfZdXCruadau40TeEdB3iZ5JfhlwyndNDv"},
        2:{"name" : "_session_public", "value" : "olUZpkrCCqkBw3acekj6dFVS0QdsonoRkIpr6AvaOTmbRDyJuu7ZpqDS5y0uD6l1YLKFKD5yOLlILQrWuUUyXIblx25OrbeCkkM9"},
        3:{"name" : "_session_public", "value" : "Vgc0HsG34m0zYQyJBvrlGukCSqvxYecOvRYHfuWwDaPEPOLELG8CPGYigjIBNzwsOOlCRQanplQpw2oYzWlysxbBJsLbmU89SjIL"},
        4:{"name" : "_session_public", "value" : "OhwTjaD9huk1qvEMmqGKdzwRNvNHQwxeBq7UKkfLZDPO2SX3ux9NizwxqTjAjpW58VO1idGnQR7ta9mvcWgOwd20tiER9hRnHaWI"},
        5:{"name" : "_session_public", "value" : "PZQNzME5q4YRqIIgzJOEPKvTvlueobznww0DcT9MoagB69SwfIuS1ZX126LurmOnMSK7FhKBGnfK6ykU37MSKoNxMe8EgZyx7En0"},
        6:{"name" : "_session_public", "value" : "Mp4pnLF8ZXyNR4nEE9AzLDTC6KpaFZ2NZ4DyLzxGPjbNL3Z4mTr5iCyWdX4Hn4INIrBVPDkHw4UdkhcgfVuYw86kCtHhC2ld4mFR"},
        7:{"name" : "_session_public", "value" : "tthelPcHx8BspcH3xbMjvKT9HwTCOP6YnxfeKVQc2AePKwKTtz9Ny4XpjbDPSrCotP0Zvl3AQPWorDNs7J3c6krZU7wSMz01HbFT"},
        8:{"name" : "_session_public", "value" : "bLYzudxmrA3ceJDZZjkxJJiG5WViJdEWgrKW2amNsTt5BBeVrZ2gjfP0jGcr3KvAGc1MkI7fI17p24jhhJJL34trZ88KsrgzM3Zp"},
        9:{"name" : "_session_public", "value" : "Nn58AG6YswSrnR42B3YXGZkDwIvDYpxjQFUZYblNvIJZEf6hQIoQckfwwsOFqf3mskhWu4q8Ajlvr7D3DSGVmuW9rfKqZnQYE05x"},
        10:{"name" : "_session_public", "value" : "ITSaUKR3L8ScvTDOYoIXOn3ofYtBCJ9YNvW54geujSi1nIL4nwfqy7ampjG41bRFdyGeySk8yGdJdlMeBR3hA7aLBaymDzRbnFgg"},
        11:{"name" : "_session_public", "value" : "vg3c4hy9Y5cFxHm4EwTK8wYksVw10xOaV8BixAH5q8XXkq2sL7BXaCr5mOCG1hlJBZPeM7xadjUWFUqI3iyG54UxmCZrXdgV9gi"},
        11:{"name" : "_session_public", "value" : "Of5freEuTmKgcl9VrrY4TOpUe885f92sollNnHmLVH4H3DN1CN79leQv2HFTqpcMKj0K547SPIIc9jtDRtS4r12I7PxXvdK8VKko"},
        }
start = time.time()
i=1
q=1
for p in datas[:2000]:
    try:
        if i%20!=0:
            print(p.get('path'))
            url = p.get('path')
            req = requests.get(url)
            content, browser = chrome(url)
            browser.add_cookie(cookies.get(q))
            q=+1
            if q>11:
                q=1
            html_r = HTML(html=content)          
            try:
                browser.find_element_by_xpath("//button[@class='btn blue mr-3 flex-item']").click()
            except NoSuchElementException:
                pass
            except ElementClickInterceptedException:
                pass
            try:
                p["name"] = element_name = browser.find_element_by_xpath("//div[@class='mb-half text lg verified']").text
            except NoSuchElementException:
                p["name"]=None
                pass
            try:
                p["age"] = browser.find_elements_by_xpath("//span[@class='text md gray-75 d-ib mr-1']")[0].text
            except NoSuchElementException:
                p["age"]=None
                pass
            try:
                p["exp"] = browser.find_elements_by_xpath("//span[@class='text md gray-75 d-ib ml-1']")[0].text
            except NoSuchElementException:
                p["exp"]=None
                pass
            try:
                p['price']= browser.find_element_by_xpath("//div[@class='flex-item fixed-size right']").text
            except NoSuchElementException:
                p['price']=None
                pass
            try:
                p['spec_lvl']=browser.find_elements_by_xpath("//span[@class='specialist--level text md']")[0].text
            except NoSuchElementException:
                p['spec_lvl']=None
                pass
            try:
                p['review-count']=browser.find_elements_by_xpath("//div[@class='flex-item fixed-size review--review-count mr-4']")[0].text
            except NoSuchElementException:
                p['review-count']=None
                pass
            try:
                p['review-positive']=browser.find_element_by_xpath("//div[@class='review--review-bars positive']//div[@class='review--review-bar']").get_attribute("style")
            except NoSuchElementException:
                p['review-positive']=None
                pass
            try:
                p['review-neutral']=browser.find_element_by_xpath("//div[@class='review--review-bars neutral']//div[@class='review--review-bar']").get_attribute("style")
            except NoSuchElementException:
                p['review-neutral']=None
                pass
            try:
                p['review-negative']=browser.find_element_by_xpath("//div[@class='review--review-bars negative']//div[@class='review--review-bar']").get_attribute("style")
            except NoSuchElementException:
                p['review-negative']=None
                pass
            try:
                p['edu_prof']=browser.find_element_by_xpath("//div[@class='text md']").text
            except NoSuchElementException:
                p['edu_prof']=None
                pass
            try:
                p['edu_university']=browser.find_element_by_xpath("//div[@class='text sm gray']").text
            except NoSuchElementException:
                p['edu_university']=None
                pass
            try:
                browser.find_element_by_xpath("//button[@class='btn blue width-100p ta-c mgb-2']").click()
                print("clicked")
                p["tel"] = browser.find_element_by_xpath("//a[@class='btn blue width-100p ta-c mgb-2']").text
                print("tel saved")
            except NoSuchElementException:
                time.sleep(5*60)
                browser.find_element_by_xpath("//button[@class='btn blue width-100p ta-c mgb-2']").click()
                p["tel"] = browser.find_element_by_xpath("//a[@class='btn blue width-100p ta-c mgb-2']").text
                pass
            except ElementClickInterceptedException:
                time.sleep(5*60)
                browser.find_element_by_xpath("//button[@class='btn blue width-100p ta-c mgb-2']").click()
                p["tel"] = browser.find_element_by_xpath("//a[@class='btn blue width-100p ta-c mgb-2']").text
                pass
            except TimeoutException:
                time.sleep(5*60)
                browser.find_element_by_xpath("//button[@class='btn blue width-100p ta-c mgb-2']").click()
                p["tel"] = browser.find_element_by_xpath("//a[@class='btn blue width-100p ta-c mgb-2']").text
                pass

        else:
            time.sleep(5*60)
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

end = time.time() - start

print(end)

https://naimi.kz/specialist/119961?service=4
clicked
https://naimi.kz/specialist/278765?service=4
clicked
https://naimi.kz/specialist/250134?service=4
clicked


KeyboardInterrupt: 

In [ ]:
datas1=datas.copy()

In [ ]:
datas

In [ ]:
with open(r'datas1.pickle', 'wb') as output:
    pickle.dump(datas1, output)

In [ ]:
print(browser.find_elements_by_xpath("//span[@class='flex-item fixed-size review--review-count mr-4']")

In [ ]:
url = 'https://naimi.kz/specialist/113762?service=4#/'
req = requests.get(url)
content, browser = chrome(url)
html_r = HTML(html=content)
browser.add_cookie({"name" : "_session_public", "value" : "EMDkrM2AsLpsnFEu3L9bkz6oS7QUnbVWyXBOW1SpMpozxRwuSTZHwkIxaia0zESLkJUK6BxDmOAv3L7yycPsWi6voEhuZkW7iB7J"})
browser.find_element_by_xpath("//button[@class='btn blue mr-3 flex-item']").click()

In [ ]:
print(browser.get_cookies())

In [ ]:
browser.add_cookie({"name" : "_session_public", "value" : "EMDkrM2AsLpsnFEu3L9bkz6oS7QUnbVWyXBOW1SpMpozxRwuSTZHwkIxaia0zESLkJUK6BxDmOAv3L7yycPsWi6voEhuZkW7iB7J"})

In [ ]:
content_blocks = browser.find_elements_by_class_name("specialist--profile")

In [ ]:
element_price = browser.find_element_by_xpath("//div[@class='flex-item fixed-size right']").text

In [ ]:
element_price

In [ ]:
browser.find_elements_by_xpath("//span[@class='specialist--level text md']")[0].text

In [ ]:
element_age = browser.find_elements_by_xpath("//span[@class='text md gray-75 d-ib mr-1']")[0].text

In [ ]:
element_exp = browser.find_elements_by_xpath("//span[@class='text md gray-75 d-ib ml-1']")[0].text

In [ ]:
element_age

In [ ]:
element_exp

In [ ]:
element_name

In [ ]:
# f = open("naimi.kz_cookies.txt", "r")
# print(f.read())

In [ ]:
element_btn = browser.find_element_by_xpath("//button[@class='btn blue width-100p ta-c mgb-2']")

In [ ]:
element_btn.text

In [ ]:
element_btn.click()

In [ ]:
element_btn_tel = browser.find_element_by_xpath("//a[@class='btn blue width-100p ta-c mgb-2']").text

In [ ]:
print(element_btn_tel)

In [ ]:
browser.find_element_by_xpath("//div[@class='review--review-bars positive']//div[@class='review--review-bar']").get_attribute("style")

In [ ]:
browser.find_element_by_xpath("//div[@class='review--review-bars negative']//div[@class='review--review-bar']").get_attribute("style")

In [ ]:
browser.find_element_by_xpath("//div[@class='review--review-bars neutral']//div[@class='review--review-bar']").get_attribute("style")

In [ ]:
browser.find_element_by_xpath("//div[@class='text md']").text

In [ ]:
browser.find_element_by_xpath("//div[@class='text sm gray']").text